In [2]:
from huggingface_hub import notebook_login
notebook_login()

In [4]:
from evaluate import load
bertscore = load("bertscore")
predictions = ["hello there", "general kenobi"]
references = ["hello there", "general kenobi"]
results = bertscore.compute(predictions=predictions, references=references, lang="en")

In [5]:
results

{'precision': [0.9999998807907104, 1.0000001192092896],
 'recall': [0.9999998807907104, 1.0000001192092896],
 'f1': [0.9999998807907104, 1.0000001192092896],
 'hashcode': 'roberta-large_L17_no-idf_version=0.3.12(hug_trans=4.28.0)'}

In [ ]:
def generate(context, model, tokenizer):
  # create prompt
  prompt = context + "agent: "

  # tokenize input (string -> tensor)
  input_ids = tokenizer(prompt).input_ids

  # call model generate (https://huggingface.co/docs/transformers/v4.34.0/en/main_classes/text_generation#transformers.GenerationMixin.generate)
  # average length of sentence for agent response is 50 -> estimate around 150 new tokens 
  output_ids = model.generate(input_ids, max_new_tokens = 150, eos_token_id = )
  
  # decode output (tensor -> string)
  pred_response = tokenizer.batch_decode(output_ids, skip_special_tokens = True)

  return pred_response

def main():
  ## load test data using pandas
  test_data = load("evaluation_csv_v1.csv")
  context_response_pairs = subsample(test_data, n=1000) 

  ## look into AutoModelForCausalLM.from_pretrained()
  model_path = "../finetuning/distilgpt2-finetuned-wikitext2/"
  model, tokenizer = load_model(model_path)
  
  # loop over the test set and collect pred, true responses into a list
  all_pred_responses, all_true_responses = [], []
  for context, true_response in context_response_pairs:
    pred_response = generate(context, model, tokenizer)
    all_pred_responses.append(pred_response)
    all_true_responses.append(true_response)
   
  # Evaluation metric: https://huggingface.co/spaces/evaluate-metric/bertscore (use the f1 scores from the results)
  bertscore = load("bertscore")
  results = bertscore.compute(pred_response_list, true_response_list)